In [1]:
from pyspark import SparkContext
sc = SparkContext('local', 'Accumulator App')

In [2]:
!head -n1 ../data/access.log

194.44.224.248 - - - [02/Apr/2017:06:25:23 +0200] "GET /track?action_name=%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F%20%C2%AB%D0%A4%D0%BE%D1%80%D0%BC%D1%83%D0%BB%D0%B0%C2%BB%3A%20%D0%9A%D1%80%D0%B8%D1%81%20%D0%A5%D0%B5%D0%BC%D1%81%D0%B2%D0%BE%D1%80%D1%82%20%D0%B2%20%D0%BD%D0%BE%D0%B2%D0%BE%D0%BC%20%D1%84%D0%B8%D0%BB%D1%8C%D0%BC%D0%B5%20%C2%AB%D0%93%D0%BE%D0%BD%D0%BA%D0%B0%C2%BB%20-%20%D0%B6%D1%83%D1%80%D0%BD%D0%B0%D0%BB%20Rolling%20Stone&idsite=559133ba45c0640fad7095cb&rec=1&r=575326&h=7&m=25&s=23&url=http%3A%2F%2Fwww.rollingstone.ru%2Fcinema%2Fevents%2F17928.html&urlref=https%3A%2F%2Fwww.google.com.ua%2F&_id=f67a211ada28b586&_idts=1491107124&_idvc=1&_idn=1&_refts=1491107124&_viewts=1491107124&_ref=https%3A%2F%2Fwww.google.com.ua%2F&cookie=1&res=750x1334&gt_ms=635 HTTP/1.1" 200 446 0.000 "http://www.rollingstone.ru/cinema/events/17928.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13F69 

In [7]:
# create RDD from text file
log_rdd = sc.textFile('../data/access.log').persist()

# count lines with 'iphone' entry
accum_iphone = sc.accumulator(0)
accum_windows = sc.accumulator(0)
accum_opera = sc.accumulator(0)

accum_windows_val = 0

def count(line):
    global accum_windows_val
    lower = line.lower()
    accum_iphone.add(lower.find('iphone') != -1)
    accum_windows.add(lower.find('windows') != -1)
    accum_opera.add(lower.find('opera') != -1)
    
    accum_windows_val += lower.find('windows') != -1
    if accum_windows_val % 10 == 0:
        print(accum_windows_val)

log_rdd.foreach(count)

print('Number of lines with \'iphone\' phrase: %d' % accum_iphone.value)
print('Number of lines with \'windows\' phrase: %d' % accum_windows.value)
print('Number of lines with \'opera\' phrase: %d' % accum_opera.value)
print('Number of lines with \'windows val\' phrase: %d' % accum_windows_val)
# TODO: count lines with 'windows' phrase
# TODO: count lines with 'opera' phrase

# HINT: use separate accumulator for each task

# TODO: use global variables instead of accumulators and compare results

Number of lines with 'iphone' phrase: 167
Number of lines with 'windows' phrase: 363
Number of lines with 'opera' phrase: 82
Number of lines with 'windows val' phrase: 0
